## Лекция 2  BM5    

### TfidfVectorizer

In [345]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
 
# инициализируем
vectorizer = TfidfVectorizer()

# составляем корпус документов
corpus = [
  'слово1 слово2 слово3',
  'слово2 слово3',
  'слово1 слово2 слово1',
  'слово4 гр'
]

# считаем
X = vectorizer.fit_transform(corpus)
 
# получится следующая структура:
#        |  слово1  |  слово2  |  слово3  |  слово4
# текст1 |   0.6    |    0.5   |   0.6    |    0
# текст2 |   0      |    0.6   |   0.8    |    0
# текст3 |   0.9    |    0.4   |   0      |    0
# текст4 |   0      |    0     |   0      |    1
 
# чтобы получить сгенерированный словарь, из приведенной структуры TfidfVectorizer
# порядок совпадает с матрицей
vectorizer.get_feature_names()  # ['слово1', 'слово2', 'слово3', 'слово4']
 
# чтобы узнать индекс токена в словаре
print(vectorizer.vocabulary_.get('гр')) # вернет 2
 
# показать матрицу
X.toarray()
 
# теперь можно быстро подсчитать вектор для нового документа
new_doc = vectorizer.transform(['слово1 слово4 слово4']).toarray()  # результат [[0.36673901, 0, 0, 0.93032387]]
 

0


## Функция ранжирования bm25

Для обратного индекса есть общепринятая формула для ранжирования *Okapi best match 25* ([Okapi BM25](https://ru.wikipedia.org/wiki/Okapi_BM25)).    
Пусть дан запрос $Q$, содержащий слова  $q_1, ... , q_n$, тогда функция BM25 даёт следующую оценку релевантности документа $D$ запросу $Q$:

$$ score(D, Q) = \sum_{i}^{n} \text{IDF}(q_i)*\frac{TF(q_i,D)*(k+1)}{TF(q_i,D)+k(1-b+b\frac{l(d)}{avgdl})} $$ 
где   
>$TF(q_i,D)$ - частота слова $q_i$ в документе $D$      
$l(d)$ - длина документа (количество слов в нём)   
*avgdl* — средняя длина документа в коллекции    
$k$ и $b$ — свободные коэффициенты, обычно их выбирают как $k$=2.0 и $b$=0.75   
$$$$
$\text{IDF}(q_i)$ - это модернизированная версия IDF: 
$$\text{IDF}(q_i) = \log\frac{N-n(q_i)+0.5}{n(q_i)+0.5},$$
>> где $N$ - общее количество документов в коллекции   
$n(q_i)$ — количество документов, содержащих $q_i$

### __Задача 1__:    
Реализуйте поиск с метрикой *TF-IDF* через умножение матрицы на вектор.
Что должно быть в реализации:
- проиндексированная база, где каждый документ представлен в виде вектора TF-IDF
- функция перевода входяшего запроса в вектор по метрике TF-IDF
- ранжирование докуменов по близости к запросу по убыванию

В качестве корпуса возьмите корпус вопросов в РПН по Covid2019. Он состоит из:
> файл **answers_base.xlsx** - база ответов, у каждого ответа есть его номер, тематика и примеры вопросов, которые могут быть заданы к этому ответу. Сейчас проиндексировать надо именно примеры вопросов в качестве документов базы. Понимаете почему?

> файл **queries_base.xlsx** - вопросы юзеров, к каждому из которых проставлен номер верного ответа из базы. Разделите эти вопросы в пропорции 70/30 на обучающую (проиндексированную как база) и тестовую (как запросы) выборки. 


In [348]:
import re
import json
import os
import numpy as np
import pymorphy2
import pickle
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from math import log
import collections
from collections import Counter

from nltk.corpus import stopwords
stop_words = stopwords.words('russian')
# stop_words.extend()

morph = pymorphy2.MorphAnalyzer()
from sklearn.feature_extraction.text import CountVectorizer



In [349]:
def tokenize_ru(sentence):
    sentence = re.sub(r'[\'"”\,\!\?\.\-\(\)\[\]\:\;\»\«\>\—]', ' ', str(sentence).rstrip("']"))
    sentence = re.sub(r'[0-9]', ' ', str(sentence))
    sentence = sentence.lower()
    tokens = sentence.split()
    tokens = [i for i in tokens if (i not in stop_words)]
    tokens = [morph.parse(i)[0].normal_form for i in tokens]
    tokens = ' '.join(tokens)
    return tokens

In [350]:
# нужно для bm25 чтобы собрать обратно-индексированный словарь и по нему считать некоторые метрики для слов

def index_m(matrix):
    np_matrix = matrix.toarray()
    dictionary = {}
    for i, word in enumerate(vectorizer.get_feature_names()):
        dictionary[word] = [int(sum(np_matrix[:, i]))] # последний элемент - порядковый номер слова в списке уникальных словб второй - crjkmrj dctuj hfp dcnhtxftnc ckjdj 
        for ind, doc in enumerate(np_matrix[:, i].tolist()):
            if doc != 0:
                dictionary[word].append(ind)
        dictionary[word].append(i)
    return dictionary

In [351]:
# перевод запросов в соответствующий вид

def to_tiidf(query, vectorizer):
    query = query.replace('\n', ' ').replace('/', ' ')
    query = tokenize_ru(query)
    vect = vectorizer.transform([query]).toarray()
    return vect 

def to_bm25(query, dict_all_words):
    vect = np.zeros((1, len(dict_all_words.keys())))
    query = query.replace('\n', ' ').replace('/', ' ')
    query = tokenize_ru(query)
    for word in query.split(' '):
        if word in dict_all_words.keys():
            vect[0, dict_all_words[word][-1]] = 1
    return vect

In [352]:
answers = pd.read_csv('answers_base.csv', encoding = 'windows-1251', sep = ';')
queries = pd.read_csv('queries_base.csv', encoding = 'windows-1251', sep = ';')

In [353]:
corpus = []
vectorizer = TfidfVectorizer()
proximity = []

ans = answers[['Номер связки','Текст вопросов']].dropna(axis = 0, how ='any')
qw = queries[['Текст вопроса', 'Номер связки\n']].dropna(axis = 0, how ='any')
qw.rename(columns={'Текст вопроса': 'Текст вопросов', 'Номер связки\n': 'Номер связки'}, inplace=True)
ans_short_q = pd.concat([ans, qw.iloc[0:int(qw.shape[0]*0.7), :]])

for question in ans_short_q['Текст вопросов']:
    question = question.replace('\n', ' ').replace('/', ' ')
    words_doc = tokenize_ru(question)
    corpus.append(words_doc)

X = vectorizer.fit_transform(corpus)
X.toarray()

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [354]:
# создаем запрос и смотрим на его сортированные близости к текстам из матрицы

queries2 = qw.iloc[int(qw.shape[0]*0.7):, 0].tolist()
new_doc = to_tiidf(queries2[5], vectorizer).T # в качестве индекса порядковый номер запроса из документа
print(queries2[5]) 

for row in X:
    proximity.append(float(row*new_doc))
number_sv = ans_short_q['Номер связки'].tolist()
dict_ans = dict(zip(proximity, number_sv))

for k in sorted(dict_ans.keys(), reverse=True):
    print (k, ':', dict_ans[k])

Вы никаким образом мне не помогли
Мне нужно обратиться в прокуратуру, что б на мое сообщение ответила по существу?
В поликлиники мне сообщили, что необходимо ждать специалистов с тестом на ковид. Их нет, звонков и путных сообщений по существу тоже нет. Если в течении в ближайшего времени со мной не свяжуться я буду вынужден не оставаться дома и обратиться в прокуратуру, что б мою заявку наконец начали рассматривать, а не писать астрактные сообщения.  

0.22210318075556265 : 308.0
0.14221185176928028 : 308.0
0.14134896084692927 : 12.0
0.14055392150513316 : 308.0
0.12562908342237214 : 308.0
0.12462404059400163 : 12.0
0.12066525445607618 : 1.0
0.12060916794256717 : 308.0
0.11758355726782976 : 45.0
0.11293926011021202 : 12.0
0.11058277673366661 : 308.0
0.1104154736779767 : 6.0
0.10815139167713891 : 308.0
0.10773248481888874 : 308.0
0.10590582343627399 : 57.0
0.10425086915451492 : 6.0
0.10215836579752081 : 257.0
0.10186131587047599 : 1.0
0.09872298411401355 : 257.0
0.09536636668792763 : 308

### __Задача 2__:    
Аналогичная задаче1 с другой метрикой 

Реализуйте поиск с метрикой *BM25* через умножение матрицы на вектор. Что должно быть в реализации:

- проиндексированная база, где каждый документ представлен в виде вектора BM25
- функция перевода входяшего запроса в вектор по метрике BM25
- ранжирование докуменов по близости к запросу по убыванию

In [355]:
# для реализации бм25, первая - считает значения в ячейках матрицы, вторая - среднее
# количество слов по всем текстам в корпусе

def bm25(tf_q_d, l, N, nq) -> float:
    TF = (tf_q_d * (k+1))/(tf_q_d + k*(1 - b + b*(l/aver)))
    IDF = log((N-nq+0.5)/(nq + 0.5))
    result = IDF*TF
    if result == 0:
        print('null')
    return result

def Average(lst):
    lst2 = []
    d = {}
    for doc in lst:
        lst2.append(len(doc.split(' ')))
    return sum(lst2) / len(lst)

In [365]:
k = 2.0
b = 0.75
aver = Average(corpus)
N = len(corpus)

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
print(len(corpus))
di = index_m(X) # обратный словарь, первое значение-сколько раз всего встретилось слово, 
# последнее значение - индекс слова

matr = np.zeros((N, len(di.keys())))

for i, doc in enumerate(corpus):
    doc = doc.split(' ')
    tf_q_d = Counter(doc)
    l = len(doc)
    try:
        for word in doc:
            matr[i, di[word][-1]] = bm25(tf_q_d[word], l, N, len(di[word])-1)
    except KeyError:
        pass
        

print(matr.shape)

1646
(1646, 5959)


In [366]:
# готовим и смотрим на запрос и его близость

new_doc = to_bm25(queries2[5], di).T
proximity = []
print(queries2[5])

# 6563
for row in matr:
    r = row.reshape(1, 5959)
    proximity.append(float(r.dot(new_doc)))
number_sv = ans_short_q['Номер связки'].tolist()
print(type(number_sv), len(proximity))


dict_ans = dict(zip(proximity, number_sv))
for k in sorted(dict_ans.keys(), reverse=True):
    print (k, ':', dict_ans[k])

Вы никаким образом мне не помогли
Мне нужно обратиться в прокуратуру, что б на мое сообщение ответила по существу?
В поликлиники мне сообщили, что необходимо ждать специалистов с тестом на ковид. Их нет, звонков и путных сообщений по существу тоже нет. Если в течении в ближайшего времени со мной не свяжуться я буду вынужден не оставаться дома и обратиться в прокуратуру, что б мою заявку наконец начали рассматривать, а не писать астрактные сообщения.  

<class 'list'> 1646
17.257326529098798 : 308.0
16.225490519184717 : 57.0
15.876224628440351 : 1.0
15.268786937164561 : 12.0
14.82551496054549 : 308.0
14.353103087248808 : 308.0
13.873734160999932 : 308.0
13.770213052196418 : 6.0
13.472817625709128 : 308.0
13.445185109920985 : 308.0
13.42637122606317 : 6.0
13.23283447417871 : 257.0
12.69161144000417 : 12.0
12.637938822354254 : 1.0
12.4661831610244 : 37.0
12.39847682775038 : 1.0
12.313784425658383 : 6.0
12.254526701669832 : 1.0
12.119632979702073 : 6.0
11.959072921010652 : 1.0
11.944051892